### Trying to solve Task 3 now!

Prepare for Euler, i.e. logging stdout and stderr to file

In [ ]:
import sys
from datetime import datetime

class Tee(object):
    def __init__(self, *files):
        self.files = files
    def write(self, obj):
        for f in self.files:
            f.write(obj)
            f.flush() # If you want the output to be visible immediately
    def flush(self) :
        for f in self.files:
            f.flush()

In [ ]:
logfile = open('logAllFD.log', 'a')

original_stderr = sys.stderr
original_stdout = sys.stdout

sys.stdout = Tee(sys.stdout, logfile)
sys.stderr = sys.stdout

print(f"Starting new run at {datetime.now()}")

Starting new run at 2021-05-12 18:02:33.770662


In [ ]:
!pip uninstall scikit-learn yellowbrick imbalanced-learn fancyimpute -y

Uninstalling scikit-learn-0.21.3:
  Successfully uninstalled scikit-learn-0.21.3


In [ ]:
!pip install scikit-learn==0.21.3

  Using cached https://files.pythonhosted.org/packages/9f/c5/e5267eb84994e9a92a2c6a6ee768514f255d036f3c8378acfa694e9f2c99/scikit_learn-0.21.3-cp37-cp37m-manylinux1_x86_64.whl


In [ ]:
import pandas as pd
import math
import operator
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import f1_score
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import OneHotEncoder
from itertools import product
import numpy as np
from sklearn.neural_network import MLPClassifier

# Not sure if it would work on euler
#trainUrl = 'https://raw.githubusercontent.com/Toroto006/iml2021/main/task3/train.csv?token=AH44KNUX4PZX2ZQVSNK7UFLATNZ64'
#testUrl = 'https://raw.githubusercontent.com/Toroto006/iml2021/main/task3/test.csv?token=AH44KNVTLV33VIFDUBUKCTTATNZ6W'

trainUrl = 'train.csv'
testUrl = 'test.csv'

In [ ]:
trainPD = pd.read_csv(trainUrl)
print(trainPD.head(3))

  Sequence  Active
0     DKWL       0
1     FCHN       0
2     KDQP       0


In [ ]:
testPD = pd.read_csv(testUrl)
print(testPD.head(3))

  Sequence
0     HWFK
1     MWPW
2     ALDV


Let's first take a more thorough look at the data.

In [ ]:
aminoAcids = {
    "R" : 1,
    "H" : 2,
    "K" : 3,
    "D" : 4,
    "E" : 5,
    "S" : 6,
    "T" : 7,
    "N" : 8,
    "Q" : 9,
    "C" : 10,
    "U" : 11,
    "G" : 12,
    "P" : 13,
    "A" : 14,
    "I" : 15,
    "L" : 16,
    "M" : 17,
    "F" : 18,
    "W" : 19,
    "Y" : 20,
    "V" : 21,
}

def really_safe_normalise_in_place(d):
    factor=1.0/math.fsum(d.values())
    for k in d:
        d[k] = d[k]*factor
    key_for_max = max(d.items(), key=operator.itemgetter(1))[0]
    diff = 1.0 - math.fsum(d.values())
    #print "discrepancy = " + str(diff)
    d[key_for_max] += diff
    return d

#aminoAcids = really_safe_normalise_in_place(aminoAcids)
aminoAcids["D"]

4

In [ ]:
permutations= []
for elm1 in aminoAcids:
  for elm2 in aminoAcids:
    for elm3 in aminoAcids: 
      for elm4 in aminoAcids:
        permutations.append(elm1+elm2+elm3+elm4)

#print(permutations)
permutDF = pd.DataFrame(permutations, columns = ['Permutations'])

In [ ]:
splitNames = ["fir", "sec", "thr", "for"]

def splitPlaces(dfs, enc=None):
  def lmdFct(df):
    return pd.Series(list(df["Sequence"]))
  #splits = dfs.apply(lambda df: [aminoAcids.get(item,item) for item in list(df["Sequence"])], axis=1, result_type='expand').rename(columns={0:"fir", 1:"sec", 2:"thr", 3:"for"})
  splits = dfs.apply(lmdFct, axis=1, result_type='expand').rename(columns={0:"fir", 1:"sec", 2:"thr", 3:"for"})
  #print(splits.shape)
  if enc is None:
    def lmdFct2(df):
      return pd.Series(list(df["Permutations"]))
    enc = OneHotEncoder()
    temp= permutDF.apply(lmdFct2, axis=1, result_type='expand').rename(columns={0:"fir", 1:"sec", 2:"thr", 3:"for"})
    enc.fit(temp)
  onehotlabels = enc.transform(splits).toarray()
  #print(onehotlabels.shape)
  new_dfs = dfs.copy()
  #print(enc.get_feature_names())
  for ind in list(enc.get_feature_names()):
    new_dfs[ind] = 0
  new_dfs[enc.get_feature_names()] = onehotlabels
  return new_dfs, enc

splitPD, enc = splitPlaces(trainPD.head(15))

#print(splitPD.head(5))

To try the idea do train and test split.

In [ ]:
random_state = 42
# split labels
def createTrainSplit():
  Xs = splitPD.drop("Sequence", axis='columns')
  train_split, test_split = train_test_split(Xs, test_size=0.2, random_state=random_state)
  train_y = train_split["Active"]
  train_X = train_split.drop("Active", axis='columns')
  test_y = test_split["Active"]
  test_X = test_split.drop("Active", axis='columns')
  print(train_X.head(5))
  print(train_y.head(5))

Lets try a RidgeClassifier:

In [ ]:
def normalSVC():
  svcInit = SVC(class_weight='balanced', random_state=random_state)
  svc = svcInit.fit(train_X, train_y)
  train_predicted = svc.predict(train_X)
  test_predicted = svc.predict(test_X)
  print(f"train score: {f1_score(train_y, train_predicted)}") # 0.855
  print(f"test score: {f1_score(test_y, test_predicted)}") # 0.805

# normalSVC()

Looks quite good, lets do a gridsearch.

In [ ]:
def findBest():
  parameters = {'kernel':('linear', 'poly', 'rbf'), 'C':[1, 10]}
  clf = GridSearchCV(svcInit, parameters)
  clf = clf.fit(train_X, train_y)
  train_predicted = clf.predict(train_X)
  test_predicted = clf.predict(test_X)
  print(f"train score: {f1_score(train_y, train_predicted)}") # 0.99229
  print(f"test score: {f1_score(test_y, test_predicted)}") # 0.88872
  print(clf.get_params()['estimator'])
  # SVC(C=1.0, break_ties=False, cache_size=200, class_weight='balanced', coef0=0.0,
  # decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
  # max_iter=-1, probability=False, random_state=42, shrinking=True, tol=0.001,
  # verbose=False)

Wow, looks good!

In [ ]:
enc = None
def loadOneHotEncodingJoblib():
  from joblib import load
  enc = load('encoder.joblib')
#loadOneHotEncodingJoblib()

def loadOneHotEncodingPickle():
  import pickle
  with open('encoder.pickle', 'rb') as f:
    enc = pickle.load(f)
#loadOneHotEncodingPickle()

In [ ]:
X, _ = splitPlaces(trainPD, enc)
y = X["Active"]
X = X.drop(["Sequence", "Active"], axis='columns')

In [ ]:
def saveOneHotEncoding():
  from joblib import dump
  dump(enc, 'encoder.joblib')
#saveOneHotEncoding()

def dumpOneHotEncodingPickle():
  import pickle
  with open('encoder.pickle', 'wb') as f:
      pickle.dump(enc, f)
dumpOneHotEncodingPickle()

In [ ]:
testedClf = SVC(C=1.0, cache_size=2000, class_weight='balanced', coef0=0.0,
     decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
     max_iter=-1, probability=False, random_state=42, shrinking=True, tol=0.001,
     verbose=False)
#testedClf = testedClf.fit(X, y)

Prepare submission data:

In [ ]:
test_Xs, _ = splitPlaces(testPD, enc)
testXs = test_Xs.drop("Sequence", axis='columns')

In [ ]:
def doSVCpredict():
  submission_predicted = testedClf.predict(testXs)
  submission_predicted = pd.DataFrame(submission_predicted, columns=["Active"])
  return submission_predicted

#submission_predicted = doSVCpredict()

Lets try an MLPClassifier again

In [ ]:
parameters={
'learning_rate': ["constant", "invscaling", "adaptive"],
'hidden_layer_sizes': [x for x in product(range(55,100,5), range(1,10,2))],
'alpha': 10.0 ** -np.arange(1, 7),
'activation': ["logistic", "relu", "tanh"]
}

def eulerGrid(parameters):
  print("Starting to fit over grid search")

  clfMLPCgrid = GridSearchCV(estimator=MLPClassifier(solver='lbfgs', random_state=random_state),param_grid=parameters,n_jobs=-1,verbose=2,cv=10)
  print(clfMLPCgrid.fit(X, y))
  print(clfMLPCgrid.best_params_)
  submission_predicted = clfMLPCgrid.predict(testXs)
  submission_predicted = pd.DataFrame(submission_predicted, columns=["Active"])
  return submission_predicted

submission_predicted = eulerGrid(parameters)

Starting to fit over grid search
Fitting 10 folds for each of 2430 candidates, totalling 24300 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.


In [ ]:
def testPredict():
  train_predicted = clfMLPCgrid.predict(train_X)
  test_predicted = clfMLPCgrid.predict(test_X)
  print(f"train score: {f1_score(train_y, train_predicted)}")
  print(f"test score: {f1_score(test_y, test_predicted)}")

In [ ]:
sys.stdout = original_stdout
sys.stderr = original_stderr
logfile.close()

In [ ]:
def defaultMLPC():
  clf = MLPClassifier(random_state=1, max_iter=300).fit(X, y)
  submission_predicted = clf.predict(testXs)
  submission_predicted = pd.DataFrame(submission_predicted, columns=["Active"])
  return submission_predicted

#submission_predicted = defaultMLPC()

In [ ]:
submission_predicted.to_csv("testOutEuler.csv", index=False, header=False)